In [14]:
# we will need these packages that you download from Bioconductor('maftools') and CRAN ('readr', 'NMF')
library(readr)
library(NMF)
library(maftools)

# change this path according to where you created the folder
#dataPath <- "C:/-=TRANSMED=-/course materials/teaching/BIOINF17/ex1/bioinf/" 
fileList <- list.files( paste0("gdac.broadinstitute.org_OV.Mutation_Packager_Oncotated_Calls.Level_3.2016012800.0.0/"))

# this is to remove the manifest file from the list
fileList <- fileList[ fileList != "MANIFEST.txt"] 

# let's create an empty list and then populate it with the files
mafList <- list()
for (i in seq_along(fileList)) 
  mafList[[i]] <- read_tsv( paste0( 
                "gdac.broadinstitute.org_OV.Mutation_Packager_Oncotated_Calls.Level_3.2016012800.0.0/",
                fileList[i]) ,comment = "#", progress = F)

#now let's aggregate the list into one data frame
maf.df <- as.data.frame(do.call(rbind, mafList))
head(maf.df)

Parsed with column specification:
cols(
  .default = col_character(),
  Entrez_Gene_Id = col_integer(),
  NCBI_Build = col_integer(),
  Start_position = col_integer(),
  End_position = col_integer(),
  Transcript_Exon = col_integer(),
  UniProt_AApos = col_integer(),
  COSMIC_total_alterations_in_gene = col_integer(),
  gc_content = col_double(),
  CCLE_ONCOMAP_total_mutations_in_gene = col_integer(),
  i_1000gp3_AC = col_integer(),
  i_1000gp3_AF = col_double(),
  i_1000gp3_AFR_AF = col_double(),
  i_1000gp3_AMR_AF = col_double(),
  i_1000gp3_AN = col_integer(),
  i_1000gp3_DP = col_integer(),
  i_1000gp3_EAS_AF = col_double(),
  i_1000gp3_EUR_AF = col_double(),
  i_1000gp3_NS = col_integer(),
  i_1000gp3_SAS_AF = col_double(),
  i_CGC_GeneID = col_integer()
  # ... with 26 more columns
)
See spec(...) for full column specifications.
Parsed with column specification:
cols(
  .default = col_character(),
  Entrez_Gene_Id = col_integer(),
  NCBI_Build = col_integer(),
  Start_position = 

Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,...,i_gc_content_full,i_gencode_transcript_name,i_gencode_transcript_status,i_gencode_transcript_tags,i_gencode_transcript_type,i_gene_type,i_havana_transcript,i_refseq_mrna_id,i_secondary_variant_classification,i_type
BAI2,576,broad.mit.edu,37,1,32202221,32202221,+,Missense_Mutation,SNP,...,0.642,BAI2-015,KNOWN,basic|CCDS,protein_coding,protein_coding,OTTHUMT00000381838.1,NM_001703,NA,Missense_Mutation
LRRC41,10489,broad.mit.edu,37,1,46751997,46751997,+,Missense_Mutation,SNP,...,0.592,LRRC41-001,KNOWN,basic|appris_principal|CCDS,protein_coding,protein_coding,OTTHUMT00000021438.1,NM_006369,NA,Missense_Mutation
ERICH3,127254,broad.mit.edu,37,1,75055494,75055494,+,Missense_Mutation,SNP,...,0.408,C1orf173-001,KNOWN,basic|appris_principal|CCDS,protein_coding,protein_coding,OTTHUMT00000026516.1,NA,NA,Missense_Mutation
SSX2IP,117178,broad.mit.edu,37,1,85128152,85128152,+,Silent,SNP,...,0.363,SSX2IP-001,KNOWN,basic|appris_principal|CCDS,protein_coding,protein_coding,OTTHUMT00000027469.1,NM_014021,NA,Silent
NTNG1,22854,broad.mit.edu,37,1,107866918,107866918,+,Missense_Mutation,SNP,...,0.478,NTNG1-006,KNOWN,basic|appris_candidate_longest|CCDS,protein_coding,protein_coding,OTTHUMT00000030340.1,NM_014917,NA,Missense_Mutation
PGLYRP3,114771,broad.mit.edu,37,1,153274927,153274927,+,Missense_Mutation,SNP,...,0.458,PGLYRP3-001,KNOWN,basic|appris_principal|CCDS,protein_coding,protein_coding,OTTHUMT00000039488.1,NM_052891,NA,Missense_Mutation


In [19]:
print(maf.df[0,])

  [1] Hugo_Symbol                                  
  [2] Entrez_Gene_Id                               
  [3] Center                                       
  [4] NCBI_Build                                   
  [5] Chromosome                                   
  [6] Start_position                               
  [7] End_position                                 
  [8] Strand                                       
  [9] Variant_Classification                       
 [10] Variant_Type                                 
 [11] Reference_Allele                             
 [12] Tumor_Seq_Allele1                            
 [13] Tumor_Seq_Allele2                            
 [14] dbSNP_RS                                     
 [15] dbSNP_Val_Status                             
 [16] Tumor_Sample_Barcode                         
 [17] Matched_Norm_Sample_Barcode                  
 [18] Match_Norm_Seq_Allele1                       
 [19] Match_Norm_Seq_Allele2                       
 [20] Tumor_

In [42]:
# You can check here and appreciate how many possible annotations one can get
# We will choose specific columns only for ease of use

selectedColumns <- c( "Hugo_Symbol", "Center", "NCBI_Build", "Chromosome",
                      "Start_position", "End_position", "Variant_Classification",
                      "Variant_Type", "Tumor_Sample_Barcode", "Mutation_Status", 
                      "Reference_Allele", "Tumor_Seq_Allele1", "Tumor_Seq_Allele2",
                      "Protein_Change", "i_dbNSFP_CADD_phred")

maf.df <- maf.df[, selectedColumns]


In [43]:
# Let's keep only mutations marked as somatic or unknown and 
# store the rest in separate data.frame
maf.germline.df <- maf.df[! maf.df$Mutation_Status %in% c("Somatic","Unknown"), ]
maf.df <- maf.df[ maf.df$Mutation_Status %in% c("Somatic","Unknown"), ]

# let's see the structure of data frame
str(maf.df)

'data.frame':	20160 obs. of  15 variables:
 $ Hugo_Symbol           : chr  "BAI2" "LRRC41" "ERICH3" "SSX2IP" ...
 $ Center                : chr  "broad.mit.edu" "broad.mit.edu" "broad.mit.edu" "broad.mit.edu" ...
 $ NCBI_Build            : int  37 37 37 37 37 37 37 37 37 37 ...
 $ Chromosome            : chr  "1" "1" "1" "1" ...
 $ Start_position        : int  32202221 46751997 75055494 85128152 107866918 153274927 157516843 224321795 248343719 28023684 ...
 $ End_position          : int  32202221 46751997 75055494 85128152 107866918 153274927 157516843 224321795 248343719 28023684 ...
 $ Variant_Classification: chr  "Missense_Mutation" "Missense_Mutation" "Missense_Mutation" "Silent" ...
 $ Variant_Type          : chr  "SNP" "SNP" "SNP" "SNP" ...
 $ Tumor_Sample_Barcode  : chr  "TCGA-04-1331-01" "TCGA-04-1331-01" "TCGA-04-1331-01" "TCGA-04-1331-01" ...
 $ Mutation_Status       : chr  "Unknown" "Somatic" "Unknown" "Somatic" ...
 $ Reference_Allele      : chr  "C" "C" "T" "A" ...
 $ Tum

In [44]:
head(maf.df)

Hugo_Symbol,Center,NCBI_Build,Chromosome,Start_position,End_position,Variant_Classification,Variant_Type,Tumor_Sample_Barcode,Mutation_Status,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Protein_Change,i_dbNSFP_CADD_phred
BAI2,broad.mit.edu,37,1,32202221,32202221,Missense_Mutation,SNP,TCGA-04-1331-01,Unknown,C,C,G,p.R1028P,17.93
LRRC41,broad.mit.edu,37,1,46751997,46751997,Missense_Mutation,SNP,TCGA-04-1331-01,Somatic,C,C,T,p.E178K,28.5
ERICH3,broad.mit.edu,37,1,75055494,75055494,Missense_Mutation,SNP,TCGA-04-1331-01,Unknown,T,T,G,p.E666A,12.41
SSX2IP,broad.mit.edu,37,1,85128152,85128152,Silent,SNP,TCGA-04-1331-01,Somatic,A,A,G,p.G245G,NA
NTNG1,broad.mit.edu,37,1,107866918,107866918,Missense_Mutation,SNP,TCGA-04-1331-01,Somatic,G,G,A,p.M87I,19.40
PGLYRP3,broad.mit.edu,37,1,153274927,153274927,Missense_Mutation,SNP,TCGA-04-1331-01,Unknown,G,G,T,p.S229Y,14.61


In [45]:
tail(maf.df)


,Hugo_Symbol,Center,NCBI_Build,Chromosome,Start_position,End_position,Variant_Classification,Variant_Type,Tumor_Sample_Barcode,Mutation_Status,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,Protein_Change,i_dbNSFP_CADD_phred
20214,C9orf78,broad.mit.edu,37,9,132597484,132597484,Missense_Mutation,SNP,TCGA-61-2113-01,Unknown,T,T,G,p.K6T,13.48
20215,CAMSAP1,broad.mit.edu,37,9,138714002,138714002,Missense_Mutation,SNP,TCGA-61-2113-01,Unknown,G,G,C,p.S835R,22.6
20216,NOTCH1,broad.mit.edu,37,9,139391081,139391081,Missense_Mutation,SNP,TCGA-61-2113-01,Unknown,G,G,T,p.S2370R,12.84
20217,DPP7,broad.mit.edu,37,9,140007856,140007856,Missense_Mutation,SNP,TCGA-61-2113-01,Unknown,G,G,A,p.A193V,17.85
20218,FAM47A,broad.mit.edu,37,X,34149721,34149721,Silent,SNP,TCGA-61-2113-01,Unknown,C,C,T,p.P225P,NA
20219,B3GNT7,broad.mit.edu,37,2,232263379,232263380,In_Frame_Ins,INS,TCGA-61-2113-01,Unknown,-,-,GCG,p.317_318insG,NA


In [13]:
write.table(maf.df, file = paste0("allSamplesMaf.maf"),
            col.names = T, row.names = F, quote = F, sep="\t")


maf <- read.maf(maf = paste0("allSamplesMaf.maf"),
                removeSilent = T, useAll = T , verbose = F)

reading maf..
Done !


In [46]:
sessionInfo()

R version 3.3.2 (2016-10-31)
Platform: i386-w64-mingw32/i386 (32-bit)
Running under: Windows 7 x64 (build 7601) Service Pack 1

locale:
[1] LC_COLLATE=Finnish_Finland.1252  LC_CTYPE=Finnish_Finland.1252   
[3] LC_MONETARY=Finnish_Finland.1252 LC_NUMERIC=C                    
[5] LC_TIME=Finnish_Finland.1252    

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] maftools_1.0.40     NMF_0.20.6          Biobase_2.34.0     
[4] BiocGenerics_0.20.0 cluster_2.0.5       rngtools_1.2.4     
[7] pkgmaker_0.22       registry_0.3        readr_1.0.0        

loaded via a namespace (and not attached):
 [1] gridExtra_2.2.1            Biostrings_2.42.1         
 [3] Rsamtools_1.26.1           stringi_1.1.2             
 [5] evaluate_0.10              memoise_1.0.0             
 [7] lattice_0.20-34            bitops_1.0-6              
 [9] RSQLite_1.1-2              survival_2.40-1           
[11] trimcluster_